In [1]:
!nvidia-smi

Mon May  9 21:05:27 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 511.79       Driver Version: 511.79       CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:2B:00.0 Off |                  N/A |
| N/A   50C    P8    N/A /  N/A |      0MiB /  2048MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [46]:
cd /content/drive/MyDrive/qianyan_similarity

/content/drive/MyDrive/qianyan_similarity


In [2]:
!pwd

D:\Develop\Project\python\qianyan_similarity\Qianyan_Sentence_Pair_Classification


In [15]:
!pip install simpletransformers

^C
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [3]:
from simpletransformers.classification import (
    ClassificationModel, ClassificationArgs
)
import pandas as pd
import logging
import csv
import tqdm
import torch

In [4]:
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [3]:
! rm -rf cache_dir/ outputs/ runs/

In [5]:
# 语料加载

corpus_name = ['lcqmc', 'bq_corpus', 'paws-x-zh']
corpus_path = corpus_name[1]


train_data = []
eval_data = []
test_data = []


with open("data/" + corpus_path + "/train.tsv", 'rt', encoding='utf-8') as fIn:
    tsv_file = csv.reader(fIn, delimiter="\t")
    for line in tsv_file:
        train_data.append(line)
train_data = train_data[:5000]
for i in range(len(train_data)):
    train_data[i][2] = int(train_data[i][2]) # 标注改为int
train_df = pd.DataFrame(train_data)
train_df.columns = ["text_a", "text_b", "labels"]


with open("data/" + corpus_path + "/dev.tsv", 'rt', encoding='utf8') as fIn:
    tsv_file = csv.reader(fIn, delimiter="\t")
    for line in tsv_file:
        eval_data.append(line)
for i in range(len(eval_data)):
    eval_data[i][2] = int(eval_data[i][2])  # 标注改为int
eval_df = pd.DataFrame(eval_data)
eval_df.columns = ["text_a", "text_b", "labels"]


with open("data/" + corpus_path + "/test.tsv", 'rt', encoding='utf8') as fIn:
    tsv_file = csv.reader(fIn, delimiter="\t")
    for line in tsv_file:
        test_data.append(line)

print(len(train_data))
print(len(eval_data))
print(len(test_data))

5000
10000
10000


In [8]:
# model_type = "bert"
# model_name = "bert-base-chinese"
model_type = "roberta"
model_name = "roberta-base"


# Optional model configuration
model_args = ClassificationArgs(num_train_epochs=1)

# Create a ClassificationModel
model = ClassificationModel(model_type, model_name, use_cuda=torch.cuda.is_available())

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifi

In [8]:
torch.cuda.is_available()

True

In [ ]:
# Train the model
model.train_model(train_df)

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/5000 [00:00<?, ?it/s]

In [ ]:
# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(
    eval_df
)

In [ ]:
# Make predictions with the model
predictions, raw_outputs = model.predict(test_data)

In [ ]:
# predictions

In [ ]:
# raw_outputs

In [ ]:
with open('result/' + corpus_path + '.tsv', 'w') as out_file:
    tsv_writer = csv.writer(out_file, delimiter='\t')
    tsv_writer.writerow(['index', 'prediction'])
    for i in range(len(predictions)):
        tsv_writer.writerow([str(i), str(predictions[i])])

In [ ]:
# model_type = "roberta"
# # model_name = "outputs/checkpoint-12500-epoch-1"
# model_name = "roberta-base"
# import torch

# # Optional model configuration
# model_args = ClassificationArgs(num_train_epochs=1)

# # Create a ClassificationModel
# model = ClassificationModel(model_type, model_name, use_cuda=torch.cuda.is_available())

In [ ]:


# # Train the model
# # model.train_model(train_df)

# # Evaluate the model
# result, model_outputs, wrong_predictions = model.eval_model(
#     eval_df
# )




In [ ]:
# # Make predictions with the model
# predictions, raw_outputs = model.predict(test_data)

In [ ]:
# predictions

In [ ]:
# raw_outputs